# Add items for human feedback

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=4072402944, available=2127781888, percent=47.8, used=1706057728, free=505847808, active=1357238272, inactive=1174188032, buffers=1638400, cached=1858859008, shared=1269760, slab=443830272)
*******************************************
YOU ARE NOT USING THE CORRECT INSTANCE TYPE
PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge 
*******************************************


In [3]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [4]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client("sagemaker", region)
comprehend = boto3.client("comprehend", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

# Retrieve the `augmented_ai_flow_definition_arn` Created Previously

In [5]:
%store -r augmented_ai_flow_definition_arn

In [6]:
print(augmented_ai_flow_definition_arn)

arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2


In [7]:
items = [
    {"prompt": "Given the following review:\nAwesome\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5", 
     "responses": [4, 5]},
    {"prompt": "Given the following review:\nSometimes it works but usually not\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5", 
     "responses": [1, 2]},
    {"prompt": "Given the following review:\nI keep buying madden every year hoping they get back to football. This years version is a little better than last years -- but that's not saying much.The game looks great. The only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />The gameplay is still slowed down by the bloated pre-play controls. What used to take two buttons is now a giant PITA to get done before an opponent snaps the ball or the play clock runs out.<br /><br />The turbo button is back, but the player movement is still slow and awkward. If you liked last years version, I'm guessing you'll like this too. I haven't had a chance to play anything other than training and a few online games, so I'm crossing my fingers and hoping the rest is better.<br /><br />The one thing I can recommend is NOT TO BUY THE MADDEN BUNDLE. The game comes as a download. So if you hate it, there's no trading it in at Gamestop.\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5", 
     "responses": [2, 3]},
    {"prompt": "Given the following review:\nIf you are prepping for the end of the world this is one of those things that you should have installed on your-end-of-the-world-proof PC.  Hail to the great Yuri!\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5",
     "responses": [4, 5]}
]

# "I enjoy this product",
# "I am unhappy with this product", 
# "It is okay", 
# "Sometimes it works but usually not",


In [8]:
human_loops_started = []

for item in items:
    print(f'Processing item: "{item}"')

    humanLoopName = str(uuid.uuid4())
#    inputContent = {"initialValue": 3, "taskObject": item}
    inputContent = {"taskObject": item}
    start_loop_response = a2i.start_human_loop(
        HumanLoopName=humanLoopName,
        FlowDefinitionArn=augmented_ai_flow_definition_arn,
        HumanLoopInput={"InputContent": json.dumps(inputContent)},
    )

    human_loops_started.append(humanLoopName)

    print(f"*** ==> Starting human loop with name: {humanLoopName}  \n")

Processing item: "{'prompt': 'Given the following review:\nAwesome\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- 1\n- 2\n- 3\n- 4\n- 5', 'responses': [4, 5]}"
*** ==> Starting human loop with name: b30ce452-0e02-4599-b486-d492e9adb965  

Processing item: "{'prompt': "Given the following review:\nI keep buying madden every year hoping they get back to football. This years version is a little better than last years -- but that's not saying much.The game looks great. The only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />The gameplay is still slowed down by the bloated pre-play controls. What used to take two buttons is now a giant PITA to get done before an opponent snaps the ball or the play clock runs out.<br /><br />The turbo button is back, but the player movement is still slow and awkward. If you liked last years version, I'm guessing you'll like this too. I haven't had a chanc

In [9]:
%store human_loops_started

Stored 'human_loops_started' (list)


# Check Status of Human Loop

In [10]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")

    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: b30ce452-0e02-4599-b486-d492e9adb965
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/b30ce452-0e02-4599-b486-d492e9adb965/output.json'}

HumanLoop Name: c7464789-5120-4b39-94d3-d950f174017c
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/c7464789-5120-4b39-94d3-d950f174017c/output.json'}

HumanLoop Name: 7fe407d0-1947-4dbf-861a-0b49b5fd5bce
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/ground-truth-star-rating-results/fd-dsoaws-star-rating-9e22832c-50fa-42c4-aa9f-507a31473ec2/2023/03/27/15/22/44/7fe407d0-1947-4dbf-861a-0b49b5fd5bce/output.json'}



# Wait For Workers to Complete Their Human Loop Tasks

Navigate to the link below and login with your email and password that you used when you set up the Private Workforce.

In [11]:
%store -r augmented_ai_workteam_arn

In [12]:
print(augmented_ai_workteam_arn)

arn:aws:sagemaker:us-east-1:079002598131:workteam/private-crowd/dsoaws


In [13]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind("/") + 1 :]
print(workteam_name)
print("Navigate to the private worker portal and complete the human loop.")
print("Make sure you have invited yourself to the workteam and received the signup email.")
print("Note:  Check your spam filter if you have not received the email.")
print("")
print("https://" + sagemaker.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"])

dsoaws
Navigate to the private worker portal and complete the human loop.
Make sure you have invited yourself to the workteam and received the signup email.
Note:  Check your spam filter if you have not received the email.

https://d0xvtcio9i.labeling.us-east-1.sagemaker.aws


# _YOU MUST LABEL THE DATA BY CLICKING THE LINK ABOVE BEFORE CONTINUING!!_

# Start Labeling

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# Select Label

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# Loop is Completed

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

# _YOU MUST LABEL THE DATA BY CLICKING THE LINK ABOVE BEFORE CONTINUING!!_

In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}
</script>